In [1]:
cd
build_path niak psom
cd /home/pbellec/git/niak_tutorials/connectome

Adding library niak to the search path.

Adding library psom to the search path.



# Subtype pipeline
This tutorial shows how to use the NIAK subtype pipeline to identify subgroups of subjects with homogeneous seed-based functional connectivity maps in the COBRE lightweight20 release, and check if some subtypes are associated with a diagnosis of schizophrenia. See the [documentation](http://niak.simexp-lab.org/pipe_connectome.html) of the pipeline for a more detailed description of the options. Download the tutorial as a notebook [here](https://raw.githubusercontent.com/SIMEXP/niak_tutorials/master/connectome/niak_tutorial_subtype_cobre.ipynb) and a matlab script [here](https://raw.githubusercontent.com/SIMEXP/niak_tutorials/master/connectome/niak_tutorial_subtype_cobre.m). To run this tutorial, we recommend to use [jupyter](http://jupyter.org/) from a niak docker container, as described in the [NIAK installation page](http://niak.simexp-lab.org/niak_installation.html). 

To run that tutorial, you need to complete first the [correlation maps tutorial](http://niak.simexp-lab.org/niak_tutorial_rmap_connectome.html). You also need to run both tutorials in the same folder. 

# Preparing the files

## Phenotypic data

We first load the phenotypic data from the COBRE sample. This command will not work if you haven't downloaded the data yet as part of the rmap tutorial. 

In [2]:
path_cobre = [pwd filesep 'cobre_lightweight20'];
file_pheno = [path_cobre filesep 'phenotypic_data.tsv.gz'];
tab = niak_read_csv_cell(file_pheno);

Now we convert the values into a series of numerical covariates, that we save in a separate .csv file:

In [3]:
list_subject = tab(2:end,1);
patient = strcmp(tab(2:end,5),'Patient');
age = str2double(tab(2:end,2)); 
FD = str2double(tab(2:end,9));
opt_csv.labels_x = list_subject; % Labels for the rows
opt_csv.labels_y = { 'age' , 'patient' , 'fd' };
niak_write_csv('model_patient.csv', [age patient FD] , opt_csv);

We specify to the pipeline where to find the model:

In [4]:
files_in.model = [pwd filesep 'model_patient.csv'];

# Connectivity maps

Now we are going to get a list of the connectivity maps associated with each subject for one network, say the DMN. Labels for each network have been specified when running the `connectome` pipeline. We simply grab the outputs of the connectome pipeline. 

In [5]:
path_connectome = [pwd filesep 'connectome'];
files_conn = niak_grab_connectome(path_connectome);
files_in.data = files_conn.rmap;

## Brain mask
We specify the mask of brain networks to the pipeline, so that it can use it to mask the grey matter. 

In [6]:
files_in.mask = files_conn.network_rois;

# Set up the options of the pipeline

First specify where to save the outputs, and how many networks to use: 

In [7]:
%% General
opt.folder_out = [pwd filesep 'subtype'];    

Then specify which covariates to use as confounds **before** the generation of subtypes. 

In [8]:
% a list of variable names to be regressed out
% If unspecified or left empty, no confounds are regressed
opt.stack.regress_conf = {'fd'};     

The options for the subtypes themselves:

In [9]:
%% Subtyping
opt.subtype.nb_subtype = 2;        % the number of subtypes to extract
opt.subtype.sub_map_type = 'mean'; % the model for the subtype maps (options are 'mean' or 'median')

Now we add an association test between subtypes and the patient label:

In [10]:
% scalar number for the level of acceptable false-discovery rate (FDR) for the t-maps
opt.association.patient.fdr = 0.05;                           
% turn on/off normalization of covariates in model (true: apply / false: don't apply)
opt.association.patient.normalize_x = false;                   
% turn on/off normalization of all data (true: apply / false: don't apply)
opt.association.patient.normalize_y = false;                  
% turn on/off adding a constant covariate to the model
opt.association.patient.flag_intercept = true;     
% To test a main effect of a variable
opt.association.patient.contrast.patient = 1; % scalar number for the weight of the variable in the contrast
opt.association.patient.contrast.fd = 0;      % scalar number for the weight of the variable in the contrast
opt.association.patient.contrast.age = 0;     % scalar number for the weight of the variable in the contrast
% type of data for visulization (options are 'continuous' or 'categorical')
opt.association.patient.type_visu = 'continuous'; 

It is also possible to add a single chi-square test on the relationship between subtypes and a categorical variable:

In [11]:
% string name of the column in files_in.model on which the contigency table will be based
opt.chi2 = 'patient';    

# Run the pipeline

In [12]:
opt.flag_test = false;  % Put this flag to true to just generate the pipeline without running it.
pipeline = niak_pipeline_subtype(files_in,opt);


Logs will be stored in /home/pbellec/git/niak_tutorials/connectome/subtype/logs/
Generating dependencies ...
   Percentage completed :  10 20 30 40 55 65 75 85 100- 0.01 sec
Setting up the to-do list ...
   I found 9 job(s) to do.
I could not find any log file. This pipeline has not been started (yet?). Press CTRL-C to cancel.
Deamon started on 29-Nov-2016 22:35:50
29-Nov-2016 22:35:50 Starting the pipeline manager...
29-Nov-2016 22:35:50 Starting the garbage collector...
29-Nov-2016 22:35:50 Starting worker number 1...

Pipeline started on 29-Nov-2016 22:35:51
user: pbellec, host: acacia, system: unix
*****************************************
29-Nov-2016 22:35:52 stack_DMN                submitted  (1 run / 0 fail / 0 done / 8 left)
29-Nov-2016 22:35:52 stack_MOTOR              submitted  (2 run / 0 fail / 0 done / 7 left)
29-Nov-2016 22:35:53 stack_DMN                finished   (1 run / 0 fail / 1 done / 7 left)
29-Nov-2016 22:35:53 subtype_DMN              submitted  (2 run / 0 fai